<a href="https://colab.research.google.com/github/fouad89/30DaysofML-Kaggle/blob/master/03_firstNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
# reading data from google drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:

# library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pandas_profiling as pp
import os

# preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# modeling
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint


# evaluation
from sklearn.metrics import mean_absolute_error

In [60]:
COMPETETION_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1"
TRAIN_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/train.csv"
TEST_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/test.csv"
OUTPUT_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/output"

In [63]:
def read_organise_data(train=TRAIN_DATA_PATH):
    """read the data from a path and splitting to features and target

    Args:
        train (path, optional): The path of training data file to. Defaults to TRAIN_DATA_PATH.

    Returns:
        X, y: X for features and y for target
    """
    full_df = pd.read_csv(train, index_col="id")
    print(f"Shape of Dataset: {full_df.shape}")
    return full_df

df= read_organise_data()

Shape of Dataset: (300000, 25)


In [ ]:
# dropping cont12
df.drop('cont12', axis=1, inplace=True)

In [65]:
y = df.target.copy()
X = df.drop('target', axis=1).copy()

In [31]:
def create_num_cols(df=X):
    return df.select_dtypes(exclude="object").columns.to_list()
def create_cat_cols(df=X):
    return [cname for cname in df.columns if (df[cname].dtypes=="object")]

num_cnames = create_num_cols(X)
cat_cnames = create_cat_cols(X)
all_cnames = num_cnames + cat_cnames

print(f"There are {len(num_cnames)} Numerical columns")
print(f"There are {len(cat_cnames)} Categorical Columns")
print(f"In total, there are {len(all_cnames)} columns")

There are 14 Numerical columns
There are 10 Categorical Columns
In total, there are 24 columns


In [32]:
# categorical columns transformation
ordinal_encode = OrdinalEncoder()
X[cat_cnames] = ordinal_encode.fit_transform(X[cat_cnames])
# scaling numerical columns
# scaler= StandardScaler()
# X[num_cnames] = scaler.fit_transform(X[num_cnames])
# X

In [33]:
# # hot encoding categorical columns
# def oneHotEncode(df=X, colNames=cat_cnames):
#     for cname in cat_cnames:
#         dummies = pd.get_dummies(df[cname], prefix=cname)
#         df = pd.concat([df, dummies], axis=1)
#         # drop the encoded col
#         df.drop([cname], axis=1, inplace=True)
#     return df

# # applying encoding
# X = oneHotEncode()


In [34]:
print(f"Shape of data after encoding: {X.shape}")

Shape of data after encoding: (300000, 24)


In [35]:
# splitting 
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [36]:
## Modeling
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))

# output layer
model.add(Dense(1, activation='linear'))
# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               3200      
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 512)              

In [37]:
# adding callback
checkpoint_name = os.path.join(OUTPUT_PATH,"weights",'Weights-{epoch:03d}--{val_loss:.5f}.hdf5') 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [38]:
#batch_size = int(X_train.shape[0] / 50)
model.fit(X, y, epochs=50,validation_split=0.2,callbacks=callbacks_list)

Epoch 1/50
7500/7500 [==============================] - 52s 7ms/step - loss: 0.6495 - mean_absolute_error: 0.6495 - val_loss: 0.5900 - val_mean_absolute_error: 0.5900

Epoch 00001: val_loss improved from inf to 0.59002, saving model to /content/drive/MyDrive/30-days-of-ml-competition1/output/weights/Weights-001--0.59002.hdf5
Epoch 2/50
7500/7500 [==============================] - 51s 7ms/step - loss: 0.5939 - mean_absolute_error: 0.5939 - val_loss: 0.5884 - val_mean_absolute_error: 0.5884

Epoch 00002: val_loss improved from 0.59002 to 0.58840, saving model to /content/drive/MyDrive/30-days-of-ml-competition1/output/weights/Weights-002--0.58840.hdf5
Epoch 3/50
7500/7500 [==============================] - 49s 7ms/step - loss: 0.5922 - mean_absolute_error: 0.5922 - val_loss: 0.5873 - val_mean_absolute_error: 0.5873

Epoch 00003: val_loss improved from 0.58840 to 0.58731, saving model to /content/drive/MyDrive/30-days-of-ml-competition1/output/weights/Weights-003--0.58731.hdf5
Epoch 4/50


In [39]:
X_test = pd.read_csv(TEST_DATA_PATH, index_col='id')
X_test[cat_cnames] = ordinal_encode.transform(X_test[cat_cnames])


In [40]:
X_test

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,4.0,8.0,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
5,0.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,7.0,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
15,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,10.0,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
16,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
17,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,5.0,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499987,1.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,4.0,8.0,0.505445,0.710839,0.225285,0.932926,0.287454,0.543800,0.682378,1.028978,1.022741,0.683903,0.877273,0.532410,0.605397,0.884581
499990,1.0,0.0,0.0,2.0,1.0,1.0,0.0,4.0,2.0,5.0,0.350751,0.887132,0.346864,0.284264,0.794881,0.432778,0.389775,0.359871,0.550013,0.492082,0.202295,0.416875,0.406205,0.758665
499991,0.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,8.0,0.378393,0.549489,0.411319,0.437275,0.514487,0.060997,0.171741,0.317185,0.150340,0.122109,0.390524,0.334026,0.378987,0.839416


In [41]:
# predictions 
predictions = model.predict(X_test)

In [57]:
def output_submission(prediction, file_name):
    """creating a kaggle submission file

    Args:
        prediction (array): an array of predictions of the test dataset
        file_name (string): a string for the name without the extension
    """
    my_submission = pd.DataFrame({'target': predictions[:,0]},
                                 index=X_test.index)
    #my_submission.set_index('id')
    file_path = os.path.join(OUTPUT_PATH,file_name)
    my_submission.to_csv(f'{file_path}.csv')
    print(f'A submission file has been made at {file_path}')

In [58]:
output_submission(predictions, "DL_submission")

A submission file has been made at /content/drive/MyDrive/30-days-of-ml-competition1/output/DL_submission


In [48]:
pd.DataFrame({'id':X_test.index, 'target':predictions})

Exception: ignored

In [50]:
predictions.shape

(200000, 1)

,target
id,
0,8.172789
5,8.284880
15,8.342461
16,8.239244
17,8.282663
...,...
499987,8.257453
499990,8.263576
499991,8.338556
